In [7]:
from pathlib import Path
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from hekasre_datamodule import HeKasreDataModule
from hekasre_model import HeKasreGruModel

In [8]:
data_module = HeKasreDataModule(
    train_path=Path("./data/batch_01/train.json"),
    val_path=Path("./data/batch_01/val.json"),
    test_path=Path("./data/batch_01/test.json"),
    fasttext_path=Path("./cc.fa.300.bin"),
    mode="pack",
)

/home/fatemeh/.local/lib/python3.6/site-packages/pytorch_lightning/core/datamodule.py:74: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7."


In [9]:
len(data_module.train_data)

673

In [10]:
len(data_module.val_data)

100

In [11]:
len(data_module.test_data)

200

In [12]:
model = HeKasreGruModel()

In [13]:
early_stopping = EarlyStopping(monitor="val_f1", patience=20, mode="max")
model_checkpoint = ModelCheckpoint(
    dirpath="checkpoints",
    filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}",
    monitor="val_f1",
    mode="max",
    save_top_k=5,
    save_last=True,
    every_n_epochs=1,
)

trainer = Trainer(
    max_epochs=200,
    auto_lr_find=True,
    callbacks=[early_stopping, model_checkpoint],
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [14]:
trainer.tune(model, data_module)

/home/fatemeh/.local/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:112: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/home/fatemeh/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
/home/fatemeh/.local/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:112: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Conside

{'lr_find': <pytorch_lightning.tuner.lr_finder._LRFinder at 0x7f21741dcdd8>}

In [15]:
trainer.fit(model, data_module)

/home/fatemeh/.local/lib/python3.6/site-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "

  | Name       | Type   | Params
--------------------------------------
0 | linear_inp | Linear | 30.2 K
1 | gru        | GRU    | 121 K 
2 | linear_out | Linear | 903   
--------------------------------------
152 K     Trainable params
0         Non-trainable params
152 K     Total params
0.609     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/fatemeh/.local/lib/python3.6/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:623: UserWarning: Checkpoint directory /home/fatemeh/projects/nlp-project/final/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Epoch 49: 100%|██████████| 26/26 [00:04<00:00,  6.16it/s, loss=0.00343, v_num=2, val_precision=0.980, val_recall=0.980, val_accuracy=0.980, val_f1=0.980, val_loss=0.0949]


/home/fatemeh/.local/lib/python3.6/site-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  f"DataModule.{name} has already been called, so it will not be called again. "


In [16]:
model.load_from_checkpoint(model_checkpoint.best_model_path)

HeKasreGruModel(
  (linear_inp): Linear(in_features=301, out_features=100, bias=True)
  (gru): GRU(100, 100, bidirectional=True)
  (linear_out): Linear(in_features=300, out_features=3, bias=True)
)

In [17]:
trainer.test(model, data_module.train_dataloader())

Testing: 0it [00:00, ?it/s]

/home/fatemeh/.local/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:454: UserWarning: Your `test_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  f"Your `{mode.dataloader_prefix}_dataloader` has `shuffle=True`,"
/home/fatemeh/.local/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:112: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Testing: 100%|██████████| 22/22 [00:00<00:00, 18.56it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.9998605847358704,
 'test_f1': 0.9998605847358704,
 'test_loss': 0.0034246868453919888,
 'test_precision': 0.9998605847358704,
 'test_recall': 0.9998605847358704}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 22/22 [00:00<00:00, 28.33it/s]


[{'test_precision': 0.9998605847358704,
  'test_recall': 0.9998605847358704,
  'test_accuracy': 0.9998605847358704,
  'test_f1': 0.9998605847358704,
  'test_loss': 0.0034246868453919888}]

In [18]:
trainer.test(model, data_module.val_dataloader())

Testing: 100%|██████████| 4/4 [00:00<00:00, 10.07it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.9801323413848877,
 'test_f1': 0.9801323413848877,
 'test_loss': 0.09486888349056244,
 'test_precision': 0.9801323413848877,
 'test_recall': 0.9801323413848877}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 4/4 [00:00<00:00, 14.74it/s]


[{'test_precision': 0.9801323413848877,
  'test_recall': 0.9801323413848877,
  'test_accuracy': 0.9801323413848877,
  'test_f1': 0.9801323413848877,
  'test_loss': 0.09486888349056244}]

In [19]:
def inference(model, text):
    mapping = {0: 0, 1: -1, 2: 1}
    return [
        mapping[n.item()]
        for n in (
            model.inference(**data_module.extract_features(text)).argmax(-1).squeeze(-1)
        )
    ]

In [20]:
def output_decoder(input_sentence, output_map):
    output_sentence = []
    i = 0
    for value in output_map:
        corresponding_word = input_sentence[i]
        i += 1
        if value == 1:
            corresponding_word = corresponding_word[0:-1]
        elif value == -1:
            corresponding_word += "ه"
        output_sentence.append(corresponding_word)
    output_sentence = " ".join(output_sentence)
    return output_sentence

In [23]:
text = "این کتاب ماله من نیست"

In [24]:
input_sentence = text.split()
output_map = inference(model, input_sentence)
output_decoder(input_sentence, output_map)

'این کتاب مال من نیست'